In [1]:
import pandas as pd
import numpy as np


In [2]:
from metrics_f1 import calc_f1_score

## Загрузка данных

In [3]:
path_train = r"./"

In [4]:
# данные по дислокации
dislok = pd.read_parquet(path_train + '/dislok_wagons.parquet').convert_dtypes()
# данные по текущим ремонтам
pr_rem = pd.read_parquet(path_train + '/pr_rems.parquet').convert_dtypes()
# список вагонов с остаточным пробегом на момент прогноза
wag_prob = pd.read_parquet(path_train + '/wagons_probeg_ownersip.parquet').convert_dtypes()
 # параметры вагона
wag_param = pd.read_parquet(path_train + '/wag_params.parquet').convert_dtypes()
# таргет по прогноза выбытия вагонов в ПР на месяц и на 10 дней
target = pd.read_csv(path_train +'/target/y_train.csv').convert_dtypes()
 # текущие ремонты вагонов
tr_rem = pd.read_parquet(path_train + '/tr_rems.parquet').convert_dtypes()

In [5]:
wag_prob['wagnum'].nunique()

33977

In [6]:
wag_param = wag_param.drop_duplicates(subset='wagnum', keep='last')# у вагонов могут меняться параметры, поэтмоу номер дублируется. В данной модели это фактор не учитывается

In [7]:
month_to_predict = pd.to_datetime('2022-12-01')

In [8]:
target.month = pd.to_datetime(target.month)
target = target[target.month == month_to_predict][['wagnum','target_month','target_day']]

In [9]:
target.target_month.sum(), target.target_day.sum()

(1584, 570)

In [10]:
target['wagnum'].nunique()

33977

# Наивная модель

Наивная модель будет построена на правилах с использованием минимального набора данных, без применения Ml.

Реальный процесс выглядит следующим образом - в начале месяца берется срез по парку по всем вагонам, за ремонт которых несёт ответственность ПГК. Для выбранных вагонов требуется установить, какие из них будут отремонтированы в текущем месяце. Данная информация помогает планировать нагрузку на вагоно-ремонтное предприятие(ВРП). Вторая модель определяет критичные вагоны, которые будут отправлены в ремонт в первую очередь( в ближайшие 10 дней). Это помогает фокусировать внимание диспетчеров.

Основными критериями по которым вагон отправляется в плановый ремонт - является его остаточный пробег и срок до планового ремонта.
В регламентах РЖД используется следующее правило - если ресурс по пробегу не превышает 500 км и/или плановый ремонт должен наступить через 15 дней(или меньше), то вагон может ехать только на ВРП.
Из этого регламента вытекают две особенности:
1. Диспетчер старается отправить вагон раньше положенных значений. Это позволяет выбрать предприятия, на которых ремонтироваться дешевле, а не ближайшее.
2. Компания-оператор может выбирать какому из нормативов нужно следовать - ремонтировать вагон по сроку, или по пробегу, или по обоим критериям сразу. Поэтому встречаются вагоны, у которых пробег может не отслеживаться.

Вагон может быть отправлен в плановый ремонт и раньше положенного. На это может влиять, например, история грузовых операций и количество текущих(мелких) ремонтов. Основная цель участников в данной задаче - найти закономерности и оценить значимые признаки, по которым вагон выбывает в плановый ремонт.

## Подготовка данных

In [11]:
wag_prob.query('wagnum == 33364')['repdate'].unique()

array(['2022-08-01T00:00:00.000000000', '2022-08-02T00:00:00.000000000',
       '2022-08-03T00:00:00.000000000', '2022-08-04T00:00:00.000000000',
       '2022-08-05T00:00:00.000000000', '2022-08-06T00:00:00.000000000',
       '2022-08-07T00:00:00.000000000', '2022-08-08T00:00:00.000000000',
       '2022-08-09T00:00:00.000000000', '2022-08-10T00:00:00.000000000',
       '2022-08-11T00:00:00.000000000', '2022-08-12T00:00:00.000000000',
       '2022-08-13T00:00:00.000000000', '2022-08-14T00:00:00.000000000',
       '2022-08-15T00:00:00.000000000', '2022-08-16T00:00:00.000000000',
       '2022-08-17T00:00:00.000000000', '2022-08-18T00:00:00.000000000',
       '2022-08-19T00:00:00.000000000', '2022-08-20T00:00:00.000000000',
       '2022-08-21T00:00:00.000000000', '2022-08-22T00:00:00.000000000',
       '2022-08-23T00:00:00.000000000', '2022-08-24T00:00:00.000000000',
       '2022-08-25T00:00:00.000000000', '2022-08-26T00:00:00.000000000',
       '2022-08-27T00:00:00.000000000', '2022-08-28

In [12]:
# оставим только данные по остаточному пробегу для каждого номерав вагона
wag_prob = wag_prob[(wag_prob.repdate == month_to_predict) | (wag_prob.repdate == wag_prob.repdate.min())]

In [13]:
wag_prob.head()

,repdate,wagnum,ost_prob,manage_type,rod_id,reestr_state,ownership_type,month
0,2022-08-01,33361,7541,0,1,1,0,8
122,2022-12-01,33361,159916,0,1,1,0,12
273,2022-08-01,33364,37103,0,1,1,0,8
395,2022-12-01,33364,4268,0,1,1,0,12
546,2022-08-01,33366,10242,0,1,1,0,8


In [14]:
wag_prob.count()

repdate           67952
wagnum            67952
ost_prob          64947
manage_type       67952
rod_id            67952
reestr_state      67952
ownership_type    67952
month             67952
dtype: int64

In [15]:
# оценим среднесуточный пробег из данных по пробегу вагона, на тот случай, если данных по нормативу нет
wag_prob_ =wag_prob.groupby('wagnum', as_index = False).agg({'repdate':['max', 'min'] , 'ost_prob': ['max','min']},)#.droplevel(1)
wag_prob_.columns = [head+'_' + name
                     if head!='wagnum'
                     else head
                     for head, name in wag_prob_.columns ]

wag_prob_['diff_days'] = wag_prob_.repdate_max - wag_prob_.repdate_min
wag_prob_['mean_run'] = (wag_prob_.ost_prob_max - wag_prob_.ost_prob_min )/ wag_prob_.diff_days.dt.days
wag_prob = wag_prob[wag_prob.repdate == wag_prob.repdate.max()][['wagnum','ost_prob']]
wag_prob = wag_prob.merge(wag_prob_[['wagnum','mean_run']])

In [16]:
wag_prob.head()

,wagnum,ost_prob,mean_run
0,33361,159916,1248.97541
1,33364,4268,269.139344
2,33366,1507,71.598361
3,33358,30223,95.114754
4,33349,153839,1214.07377


In [17]:
wag_prob[wag_prob['ost_prob'].isna()]

,wagnum,ost_prob,mean_run
72,2052,<NA>,<NA>
85,2035,<NA>,<NA>
128,1652,<NA>,<NA>
132,1956,<NA>,<NA>
133,2360,<NA>,<NA>
...,...,...,...
33833,18737,<NA>,<NA>
33890,2151,<NA>,<NA>
33916,2098,<NA>,<NA>
33917,2186,<NA>,<NA>


In [18]:
# для каждого вагона оставим только информацию по сроку службы и нормативу суточного пробега между ПР
wag_param = wag_param[['wagnum','srok_sl','cnsi_probeg_dr','cnsi_probeg_kr']]

In [19]:
wag_param.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr
3218,26318,2022-04-27,160,160
19128,28344,2024-12-24,110,160
21526,8099,2027-10-01,110,160
32353,33350,2047-02-05,250,500
81,5308,2027-09-28,110,160


In [20]:
# добавим признак, что вагон был в ПР в предыдущем месяце. Скорее всего, если вагон был в ПР недавно, то повторно он не поедет
pr_rem['was_repair_in_prev_month'] = 1
pr_rem = pr_rem[['wagnum','was_repair_in_prev_month']]
pr_rem = pr_rem.drop_duplicates(subset='wagnum') #некоторые вагоны все же ремонтируются больше 1 раза, поэтому нужен сбросить дубли

In [21]:
pr_rem.tail()

,wagnum,was_repair_in_prev_month
1577,9083,1
1578,11193,1
509,24703,1
511,26437,1
1515,15740,1


In [22]:
# посчитаем сколько текущих ремонтов было за прошедший период
tr_rem = tr_rem.groupby('wagnum', as_index= False).kod_vrab.count()

In [23]:
tr_rem.tail()

,wagnum,kod_vrab
17844,33963,2
17845,33969,2
17846,33973,2
17847,33975,2
17848,33976,2


In [24]:
# сохраним только дату следующего планового ремонта для вагона
dislok = dislok[['wagnum','date_pl_rem']].drop_duplicates(subset = 'wagnum', keep='last')

In [25]:
dislok.head()

,wagnum,date_pl_rem
347,11219,2019-06-27
25426,33350,2022-07-25
216913,8099,2023-04-20
730409,28344,2022-11-30
993683,26318,2023-01-01


In [26]:
# соберем все данные вместе
wp = target[['wagnum']].merge(wag_param, on ='wagnum', how = 'left')\
             .merge(wag_prob, how = 'left')\
             .merge(pr_rem, how = 'left')\
             .merge(tr_rem, how = 'left')\
             .merge(dislok, how = 'left')

In [27]:
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem
0,33361,2033-03-01,110,160,159916,1248.97541,1.0,3.0,2023-02-17
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23
4,33349,2033-12-04,110,160,153839,1214.07377,1.0,NaN,2023-07-06


In [28]:
# Получим среднесуточный пробег, как среднее от нормативов и реального пробега
wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']] = wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']].fillna(0)
wp['day_run'] = wp.apply(lambda x : [ val  for val in [x.cnsi_probeg_kr, x.cnsi_probeg_dr, x.mean_run] if val != 0], axis = 1 )

In [29]:
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem,day_run
0,33361,2033-03-01,110,160,159916,1248.97541,1.0,3.0,2023-02-17,"[160, 110, 1248.9754098360656]"
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03,"[160, 110, 269.1393442622951]"
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03,"[160, 110, 71.59836065573771]"
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23,"[160, 110, 95.11475409836065]"
4,33349,2033-12-04,110,160,153839,1214.07377,1.0,NaN,2023-07-06,"[160, 110, 1214.0737704918033]"


In [30]:
wp['day_run']= wp.apply(lambda x : np.mean(x.day_run) if len(x.day_run)> 0 else 0, axis = 1 )

In [31]:
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem,day_run
0,33361,2033-03-01,110,160,159916,1248.97541,1.0,3.0,2023-02-17,506.325137
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03,179.713115
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03,113.866120
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23,121.704918
4,33349,2033-12-04,110,160,153839,1214.07377,1.0,NaN,2023-07-06,494.691257


In [32]:
wp['current_date'] = month_to_predict

In [33]:
# определим, сколько дней осталось до истечения срока службы
wp['date_diff_srk_sl'] = wp['srok_sl']- wp['current_date']

In [34]:
# определим, сколько дней осталось до ближайшего ПР
wp['date_diff_pl_rem'] = wp['date_pl_rem']- wp['current_date']

In [35]:
# определим, какой остаточный ресурс будет на момент окончания месяца
wp['prob_end_month'] = wp['ost_prob'] - wp['day_run']* 30

In [36]:
wp['target_month'] = 0

In [37]:
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem,day_run,current_date,date_diff_srk_sl,date_diff_pl_rem,prob_end_month,target_month
0,33361,2033-03-01,110,160,159916,1248.97541,1.0,3.0,2023-02-17,506.325137,2022-12-01,3743 days,78 days,144726.245902,0
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03,179.713115,2022-12-01,3054 days,306 days,-1123.393443,0
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03,113.866120,2022-12-01,3338 days,123 days,-1908.983607,0
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23,121.704918,2022-12-01,3652 days,449 days,26571.852459,0
4,33349,2033-12-04,110,160,153839,1214.07377,1.0,NaN,2023-07-06,494.691257,2022-12-01,4021 days,217 days,138998.262295,0


In [38]:
pd.to_timedelta('40 days')

Timedelta('40 days 00:00:00')

In [39]:
# вагон выбывает в ПР в следующем месяце, если:
# остаточный пробег < 5 000 км
# срок службы < 500 лней
# до следующего  ПР < 40 дней
# ,число текущих ремонтов > 5
wp.loc[(wp.prob_end_month <= 5000) \
       | (wp.date_diff_srk_sl < pd.to_timedelta('500 days'))\
        | (wp.date_diff_pl_rem < pd.to_timedelta('40 days')) \
        | (wp.kod_vrab > 5),'target_month'] = 1

In [40]:
wp['target_day'] = wp['target_month']

In [41]:
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem,day_run,current_date,date_diff_srk_sl,date_diff_pl_rem,prob_end_month,target_month,target_day
0,33361,2033-03-01,110,160,159916,1248.97541,1.0,3.0,2023-02-17,506.325137,2022-12-01,3743 days,78 days,144726.245902,0,0
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03,179.713115,2022-12-01,3054 days,306 days,-1123.393443,1,1
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03,113.866120,2022-12-01,3338 days,123 days,-1908.983607,1,1
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23,121.704918,2022-12-01,3652 days,449 days,26571.852459,0,0
4,33349,2033-12-04,110,160,153839,1214.07377,1.0,NaN,2023-07-06,494.691257,2022-12-01,4021 days,217 days,138998.262295,0,0


In [42]:
pred_target = target[['wagnum']].merge(wp[['wagnum','target_month','target_day']],how = 'left')
pred_target = pred_target.drop_duplicates(subset = 'wagnum')

In [43]:
pred_target.head()

,wagnum,target_month,target_day
0,33361,0,0
1,33364,1,1
2,33366,1,1
3,33358,0,0
4,33349,0,0


In [44]:
# Проверим соотношение отмеченных вагонов с фактическим значением
round(pred_target.target_month.sum() / target.target_month.sum(), 2)

4.08

In [45]:
# сохраним таргет за месяц  для выбранного периода отдельно
target_path = './prediction/target_predicton.csv'

In [46]:
pred_target.drop_duplicates(subset = 'wagnum').to_csv(target_path, index=False)

In [47]:
true_target_path = './prediction/target_predicton_true.csv'

In [48]:
target.drop_duplicates(subset = 'wagnum').to_csv(true_target_path, index=False)

In [49]:
# оценим насколько хорошо удалось предсказать выбытие вагонов  по месяцу и по 10 дням
calc_f1_score( true_target_path, target_path,)

0.24597196903853533